# Fucntionality for reproducible Auditing in Wiggum

In [1]:
import pandas as pd
import os
import wiggum as wg
import numpy as np

In [2]:
labeled_df = wg.LabeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [3]:
rankobj = wg.Mean_Rank_Trend()
linreg_obj = wg.All_Linear_Trend()

labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/trend_components/regression.py:119: UserWarning: cannot compute with two different weights
  warnings.warn('cannot compute with two different weights')


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type
386,num_stops,search_conducted_rate,driver_race,Other,0.00508864,62.781159,lin_reg,0.0184825,82.467302,aggregate-subgroup
154,num_stops,driver_gender,state,RI,"[F, M]",0.091600,rank_trend,"[F, M]",0.116400,aggregate-subgroup
87,contraband_found_rate,driver_race,state,WI,"[Asian, White, Hispanic, Other, Black]",0.142000,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.071600,aggregate-subgroup
13,search_conducted_rate,driver_gender,driver_race,Asian,"[F, M]",0.393500,rank_trend,"[F, M]",0.251900,aggregate-subgroup
377,num_stops,search_conducted_rate,state,TX,0.0162772,323.537290,lin_reg,0.0184825,82.467302,aggregate-subgroup
88,contraband_found_rate,driver_race,driver_gender,F,"[Asian, White, Hispanic, Black, Other]",0.207700,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.071600,aggregate-subgroup
37,search_conducted_rate,driver_race,state,WA,"[Asian, White, Hispanic, Black, Other]",0.528600,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,aggregate-subgroup
244,num_stops,state,driver_race,Black,"[VT, RI, WI, CO, CT, AZ, MA, WA, MD, IL, NC, T...",0.768800,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900,aggregate-subgroup
72,contraband_found_rate,driver_gender,recreational,1,"[F, M]",0.237500,rank_trend,"[F, M]",0.179700,aggregate-subgroup
100,hit_rate,driver_gender,state,CT,"[F, M]",0.180600,rank_trend,"[F, M]",0.043300,aggregate-subgroup


In [4]:
labeled_df.result_df.iloc[128]

feat1                                                    hit_rate
feat2                                                 driver_race
group_feat                                                  state
subgroup                                                       MA
subgroup_trend             [Asian, Other, Hispanic, Black, White]
subgroup_trend_strength                                    0.0755
trend_type                                             rank_trend
agg_trend                  [Asian, Hispanic, Other, Black, White]
agg_trend_strength                                         0.0023
comparison_type                                aggregate-subgroup
Name: 128, dtype: object

In [5]:
labeled_df.annotate(128,'flagged',1)

feat1                                                    hit_rate
feat2                                                 driver_race
group_feat                                                  state
subgroup                                                       MA
subgroup_trend             [Asian, Other, Hispanic, Black, White]
subgroup_trend_strength                                    0.0755
trend_type                                             rank_trend
agg_trend                  [Asian, Hispanic, Other, Black, White]
agg_trend_strength                                         0.0023
comparison_type                                aggregate-subgroup
flagged                                                         1
Name: 128, dtype: object

We can confirm it works by checking with iloc

In [6]:
labeled_df.result_df.iloc[128]

feat1                                                    hit_rate
feat2                                                 driver_race
group_feat                                                  state
subgroup                                                       MA
subgroup_trend             [Asian, Other, Hispanic, Black, White]
subgroup_trend_strength                                    0.0755
trend_type                                             rank_trend
agg_trend                  [Asian, Hispanic, Other, Black, White]
agg_trend_strength                                         0.0023
comparison_type                                aggregate-subgroup
flagged                                                         1
Name: 128, dtype: object

In [7]:
labeled_df.filter_annotate(feat1='year',subgroup=['Black','Hispanic'],annotate_col='target', comment = 1)

4  total rows meet the criteria


/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/auditing_reporting.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_df[annotate_col] = comment


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type,flagged,target
272,year,num_stops,driver_race,Black,-2262.72,0.169689,lin_reg,-1560.86,0.043965,aggregate-subgroup,,1
273,year,num_stops,driver_race,Hispanic,422.667,0.014732,lin_reg,-1560.86,0.043965,aggregate-subgroup,,1
300,year,search_conducted_rate,driver_race,Black,1.24021,0.001744,lin_reg,1.96425,0.004473,aggregate-subgroup,,1
301,year,search_conducted_rate,driver_race,Hispanic,5.34096,0.017419,lin_reg,1.96425,0.004473,aggregate-subgroup,,1


In [8]:
labeled_df.result_df.iloc[[272,273,300,301]]

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type,flagged,target
272,year,num_stops,driver_race,Black,-2262.72,0.169689,lin_reg,-1560.86,0.043965,aggregate-subgroup,,1
273,year,num_stops,driver_race,Hispanic,422.667,0.014732,lin_reg,-1560.86,0.043965,aggregate-subgroup,,1
300,year,search_conducted_rate,driver_race,Black,1.24021,0.001744,lin_reg,1.96425,0.004473,aggregate-subgroup,,1
301,year,search_conducted_rate,driver_race,Hispanic,5.34096,0.017419,lin_reg,1.96425,0.004473,aggregate-subgroup,,1
